In [1]:
import torch
from datasets import load_mnist, get_observation_pixels
import numpy as np
import matplotlib.pyplot as plt

BATCH_SIZE = 64
train_loader, test_loader, val_loader = load_mnist(BATCH_SIZE)

In [2]:
import pytorch_lightning as pl
from models import LabelConditionalVAE

class LabelSCVAE(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.LabelConditionalVAE = LabelConditionalVAE()
    
    def forward(self, x, y):
        return self.LabelConditionalVAE(x, y)
    
    def training_step(self, batch, batch_idx):
        x, x_cond, y = batch
        output, z_mean, z_log_var, z = self.LabelConditionalVAE(x, y)
        recon_loss, kl_loss, loss = self.LabelConditionalVAE.loss(x, output, z_mean, z_log_var)
        self.log('train_loss', loss)
        self.log('train_recon_loss', recon_loss)
        self.log('train_kl_loss', kl_loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, x_cond, y = batch
        output, z_mean, z_log_var, z = self.LabelConditionalVAE(x, y)
        recon_loss, kl_loss, loss = self.LabelConditionalVAE.loss(x, output, z_mean, z_log_var)
        self.log('val_loss', loss)
        self.log('val_recon_loss', recon_loss)
        self.log('val_kl_loss', kl_loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, x_cond, y = batch
        output, z_mean, z_log_var, z = self.LabelConditionalVAE(x, y)
        recon_loss, kl_loss, loss = self.LabelConditionalVAE.loss(x, output, z_mean, z_log_var)
        self.log('test_loss', loss)
        self.log('test_recon_loss', recon_loss)
        self.log('test_kl_loss', kl_loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [3]:
from pytorch_lightning.callbacks import RichProgressBar

from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='label_scvae_{epoch:02d}',
    save_top_k=3,
    mode='min',
)

model = LabelSCVAE()


trainer = pl.Trainer(accelerator='gpu', devices=[5], max_epochs=40, enable_progress_bar=True, callbacks=[checkpoint_callback])
trainer.fit(model, train_loader, val_loader)
#save
trainer.save_checkpoint('checkpoints/label_scvae_40.ckpt')

/Home/siv34/edzak2974/.conda/envs/pytorch_birget/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /Home/siv34/edzak2974/.conda/envs/pytorch_birget/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Home/siv34/edzak2974/.conda/envs/pytorch_birget/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /Home/siv34/edzak2974/.conda/envs/pytorch_birget/lib ...
  rank_zero_warn(
You are using a CUDA device ('A100-SXM

┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                ┃ Type                ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ LabelConditionalVAE │ LabelConditionalVAE │  979 K │
└───┴─────────────────────┴─────────────────────┴────────┘

Trainable params: 979 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 979 K                                                                                                
Total estimated model params size (MB): 3

Output()